In [4]:
try :
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [7]:
train_dir = "./cats_and_dogs_small/train"
train_cats_dir = os.path.join(train_dir, "cats")
train_dogs_dir = os.path.join(train_dir, "dogs")

test_dir = "./cats_and_dogs_small/test"
test_cats_dir = os.path.join(test_dir, "cats")
test_dogs_dir = os.path.join(test_dir, "dogs")

val_dir = "./cats_and_dogs_small/val"
val_cats_dir = os.path.join(val_dir, "cats")
val_dogs_dir = os.path.join(val_dir, "dogs")

In [8]:
#데이터 전처리
# 1/255 스케일
train_datagen = ImageDataGenerator(rescale=1./255,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True)

#테스트는 크기만 변환해주고 변환을 해주지 않는다. 테스트니깐
test_datagen = ImageDataGenerator(rescale=1./255)

#이미지를 디렉토리로 부터 가져오는 함수
#가져온 다음에 train_datagen을 통해서 생성해줌
#target size
train_generator = train_datagen.flow_from_directory(train_dir,
                                                   target_size=(150,150),
                                                   batch_size=20,
                                                   class_mode = "binary")

validation_generator = test_datagen.flow_from_directory(
val_dir,
target_size=(150,150),
batch_size=20,
class_mode="binary")

test_generator = test_datagen.flow_from_directory(
test_dir,
target_size=(150,150),
batch_size=20,
class_mode="binary")

Found 1689 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [4]:
for d , l in train_generator:
    print("배치 데이터 크기: ", d.shape)
    print("배치 레이블 크기: ", l.shape)
    break

배치 데이터 크기:  (20, 150, 150, 3)
배치 레이블 크기:  (20,)


In [5]:
model = tf.keras.Sequential(
[
    tf.keras.layers.Conv2D(input_shape=(150,150,3), kernel_size=(3,3),filters=32),
    
    tf.keras.layers.MaxPool2D(strides=(2,2)),
                        
    tf.keras.layers.Conv2D(kernel_size=(3,3),filters=64),
                              
    tf.keras.layers.MaxPool2D(strides=(2,2)),
                              
    tf.keras.layers.Conv2D(kernel_size=(3,3),filters=128),                          
                              
    tf.keras.layers.Flatten(),
                              
    tf.keras.layers.Dense(units=128,activation="relu"),
                              
    tf.keras.layers.Dropout(rate=0.3),
    tf.keras.layers.Dense(units=1,activation="sigmoid")]
)

model.compile(optimizer=tf.keras.optimizers.Adam(), loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()
#20개씩 100번 하는 것
history = model.fit(train_generator, steps_per_epoch=100, epochs=10, validation_data=validation_generator, validation_steps=50)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
flatten (Flatten)            (None, 147968)            0         
_________________________________________________________________
dense (Dense)                (None, 128)               1

In [6]:
model.evaluate(test_generator,verbose=0)

[5.960464477539063e-08, 0.5]